In [1]:
import os
import pandas as pd
import requests
from io import StringIO
from datetime import datetime

# Get the current working directory 
current_directory = os.getcwd() 

# Print the current working directory 
print(current_directory)

# Define the directory containing the files 
path_NOAA = current_directory+"\\Data\\NOAA\\"
path_NSRDB = current_directory+"\\Data\\NSRDB_Yearly\\"
print(path_NOAA, path_NSRDB)

dataframes = []

filename_NOAA = path_NOAA + 'HADISDH_NOAA_Weather_Station_Matches_Yearly.csv'

weather_stations_df = pd.read_csv(filename_NOAA) #read weather station data into a dataframe

print(weather_stations_df)

for filename_NSRDB in os.listdir(path_NSRDB):
    file_path = os.path.join(path_NSRDB, filename_NSRDB)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            content_df = pd.read_csv(file)
            content_df['file_id'] = int(content_df['file_id'].values[0])
            content_df['region'] = int(content_df['region'].values[0])
            content_df['water'] = int(content_df['water'].values[0])
            # Merge data frames for HADISDH/NSRDB and NOAA data
            result_df = pd.merge(content_df, weather_stations_df, on='file_id', how='left')
            result_df = result_df.drop(['HADISDH_Latitude','HADISDH_Longitude', 'NOAA_Latitude', 'NOAA_Longitude'], axis=1)
            result_df = result_df.drop(['NSRDB_latitude','NSRDB_longitude', 'Distance', 'distance_h_NSRDB'], axis=1)
            result_df['WMO_ID'] = result_df['WMO_ID'].astype(str)
            result_df['NOAA_file_csv'] = (result_df['WMO_ID'].values[0])[:6]+(result_df['WMO_ID'].values[0])[7:]+".csv"
            result_df['DATE'] = result_df['time']
            result_df['time'] = pd.to_datetime(result_df['time'])
            time2 = datetime.strptime('2006-01-01','%Y-%m-%d')
            result_df = result_df[result_df['time'] < time2]
            if result_df.loc[0, "NOAA_file_csv"] == '72261899999.csv':
                for j in range(2557, 2922):
                    result_df.loc[j, "NOAA_file_csv"] = '72261823091.csv' # file no. 6963 has a different WMO ID for the year 2005 than previous years
            print(result_df)
            NOAA_dataframes = []
            for i in range(8):
                year = 1998 + i # years 1998-2005
                url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/'+ str(year) +'/'+ result_df['NOAA_file_csv'].values[0]
                if url == 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2005/72261899999.csv':
                    url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2005/72261823091.csv'
                    print(str(year)+": File: "+result_df['NOAA_file_csv'].values[2900])
                else:
                    print(str(year)+": File: "+result_df['NOAA_file_csv'].values[0])
                response = requests.get(url)
                print(url)

                if response.status_code == 200:
                    content = response.text
                    content_df = pd.read_csv(StringIO(content))
                    content_df = content_df.drop(['LATITUDE', 'LONGITUDE', 'ELEVATION'], axis=1)
                    NOAA_dataframes.append(content_df)    
                else:
                    print(f"Failed to retrieve the file. Status code: {response.status_code}")
            combined_df = pd.concat(NOAA_dataframes)
            result2_df = pd.merge(result_df, combined_df, on='DATE', how='left')
            print(result2_df)
            dataframes.append(result2_df)

C:\Users\User
C:\Users\User\Data\NOAA\ C:\Users\User\Data\NSRDB_Yearly\
    file_id  HADISDH_Latitude  HADISDH_Longitude        WMO_ID  NOAA_Latitude  \
0      6678            30.393            -86.467  722069-99999         30.400   
1      6687            32.899            -80.041  722080-13880         32.899   
2      6772            34.272            -83.830  722185-99999         34.267   
3      6783            33.354            -84.569  722197-99999         33.357   
4      6950            33.209            -97.199  722589-99999         33.201   
5      6963            30.912           -102.917  722618-99999         30.917   
6      7239            36.311           -119.623  723898-99999         36.317   
7      7280            39.873            -75.227  724080-13739         39.873   
8      7311            39.044            -84.672  724210-93814         39.044   
9      7321            38.365            -82.555  724250-03860         38.365   
10     7329            41.563        

In [2]:
combined_all_df = pd.concat(dataframes)

In [3]:
# Write DataFrame to CSV
combined_all_df.to_csv('Daily_Data_All_Yearly.csv', index=False)